In [13]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
from datetime import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score

In [14]:
# Parameters
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
COLUMNS = ['Close', 'SMA200', 'SMA13', 'RSI14']
LABEL = 'signal'
batch_size = 1024
epochs = 100
dropout = 0.4
learning_rate = 0.01
random_state = 13
train_test_ratio = 0.2



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
data = pd.read_csv(FILE_PATH)
data = data.dropna()
# data = data.round(4)
data['signal'] = data['signal'] - 1

In [16]:
# Select the specified columns
X = data[COLUMNS].values
y = data[LABEL].values

In [17]:
# Normalize the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_test_ratio, random_state=random_state)

In [18]:
# Convert data to PyTorch Dataset
class ForexDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # Use long for multiclass labels
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [19]:
train_dataset = ForexDataset(X_train, y_train)
test_dataset = ForexDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(16 * 1, 3)  # Output 3 classes for multiclass classification
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        return x


In [21]:
model = CNN1D().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# Training phase
for epoch in tqdm(range(epochs), desc="Training Epochs"):
    model.train()
    epoch_loss = 0
    correct_train = 0
    total_train = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(2).to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += y_batch.size(0)
        correct_train += (predicted == y_batch).sum().item()
    
    train_accuracy = correct_train / total_train * 100
    
    # Evaluation phase
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.unsqueeze(2).to(device)
            y_batch = y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total_test += y_batch.size(0)
            correct_test += (predicted == y_batch).sum().item()
    
    test_accuracy = correct_test / total_test * 100
    
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    MODEL_SAVEPATH = f"D:/Programing/AI Trader/Model/cnnModelv.1.0/Model v.1.1_loss {epoch_loss:.4f}_Acc {train_accuracy:.4f}-{test_accuracy}_at {timestamp}.model"
    torch.save(model.state_dict(), MODEL_SAVEPATH)
    
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}% | @ {timestamp}')


Training Epochs:   1%|          | 1/100 [00:03<05:35,  3.39s/it]

Epoch 1/100, Loss: 143.7034, Train Accuracy: 57.99%, Test Accuracy: 57.93% | @ 2024-12-27 16:15:31


Training Epochs:   2%|▏         | 2/100 [00:05<04:08,  2.54s/it]

Epoch 2/100, Loss: 143.9451, Train Accuracy: 57.82%, Test Accuracy: 58.04% | @ 2024-12-27 16:15:33


Training Epochs:   3%|▎         | 3/100 [00:09<05:17,  3.28s/it]

...

Epoch 98/100, Loss: 143.5375, Train Accuracy: 58.09%, Test Accuracy: 57.84% | @ 2024-12-27 16:21:49


Training Epochs:  99%|█████████▉| 99/100 [06:25<00:03,  3.42s/it]

Epoch 99/100, Loss: 143.3565, Train Accuracy: 58.18%, Test Accuracy: 57.77% | @ 2024-12-27 16:21:53


Training Epochs: 100%|██████████| 100/100 [06:28<00:00,  3.89s/it]

Epoch 100/100, Loss: 143.3984, Train Accuracy: 58.19%, Test Accuracy: 58.31% | @ 2024-12-27 16:21:56
